In [1]:
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer

model_path="/data/download-model/gemma-3-1b-it"

/home/adminad/zhangdw/workspace/uv/transformers/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = AutoModelForCausalLM.from_pretrained(model_path,
                                             attn_implementation="eager",
                                             torch_dtype="auto", 
                                             device_map="cuda:3")
model

Gemma3ForCausalLM(
  (model): Gemma3TextModel(
    (embed_tokens): Gemma3TextScaledWordEmbedding(262144, 1152, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x Gemma3DecoderLayer(
        (self_attn): Gemma3Attention(
          (q_proj): Linear(in_features=1152, out_features=1024, bias=False)
          (k_proj): Linear(in_features=1152, out_features=256, bias=False)
          (v_proj): Linear(in_features=1152, out_features=256, bias=False)
          (o_proj): Linear(in_features=1024, out_features=1152, bias=False)
          (q_norm): Gemma3RMSNorm((256,), eps=1e-06)
          (k_norm): Gemma3RMSNorm((256,), eps=1e-06)
        )
        (mlp): Gemma3MLP(
          (gate_proj): Linear(in_features=1152, out_features=6912, bias=False)
          (up_proj): Linear(in_features=1152, out_features=6912, bias=False)
          (down_proj): Linear(in_features=6912, out_features=1152, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma3RMSNorm((11

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer

GemmaTokenizerFast(name_or_path='/data/download-model/gemma-3-1b-it', vocab_size=262144, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<bos>', 'eos_token': '<eos>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'boi_token': '<start_of_image>', 'eoi_token': '<end_of_image>', 'image_token': '<image_soft_token>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<eos>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("<bos>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("<mask>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=False),
	5: AddedToken

In [4]:
model_inputs = tokenizer(["The secret to baking a good cake is "], return_tensors="pt").to("cuda:3")

In [5]:
generated_ids = model.generate(**model_inputs, max_length=30)
tokenizer.batch_decode(generated_ids)[0]

/home/adminad/zhangdw/workspace/uv/transformers/lib/python3.11/site-packages/torch/_inductor/compile_fx.py:236: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
  warnings.warn(


"<bos>The secret to baking a good cake is 100% love. That's what they all say.\n\nBut seriously, baking a"

In [6]:
from datasets import load_dataset
dataset = load_dataset("json",data_files="/data/dataset/procurement.json")
dataset

DatasetDict({
    train: Dataset({
        features: ['output', 'instruction', 'input', 'system'],
        num_rows: 660
    })
})

In [7]:
dataset['train'][9]

{'output': '甲方延期支付服务费时，违约金按以下规则计算：每延期一日需向乙方支付合同总价的1‰作为违约金。若逾期超过10个工作日，乙方有权单方面解除合同。违约金计算以合同总价为基数，按日累计，且解除合同的权限与违约金支付义务并行存在。',
 'instruction': '甲方延期支付服务费时，违约金如何计算',
 'input': '',
 'system': ''}

In [8]:
dataset = dataset["train"].train_test_split(test_size=0.1, seed=42)
dataset

DatasetDict({
    train: Dataset({
        features: ['output', 'instruction', 'input', 'system'],
        num_rows: 594
    })
    test: Dataset({
        features: ['output', 'instruction', 'input', 'system'],
        num_rows: 66
    })
})

In [9]:
def tokenize_dataset(examples):
    # 编码输入文本（instruction）
    tokenized_inputs = tokenizer(
        examples["instruction"],
        padding="max_length",
        truncation=True,
        max_length=2048,
        return_special_tokens_mask=True,
    )
    
    # 编码目标文本（output）作为labels
    tokenized_labels = tokenizer(
        examples["output"],
        padding="max_length",
        truncation=True,
        max_length=2048,
        return_special_tokens_mask=True,
    )
    
    # 将编码后的output设置为labels
    tokenized_inputs["labels"] = tokenized_labels["input_ids"].copy()
    return tokenized_inputs

In [10]:
dataset = dataset.map(tokenize_dataset, batched=True)
dataset

Map: 100%|██████████| 66/66 [00:00<00:00, 557.32 examples/s]


DatasetDict({
    train: Dataset({
        features: ['output', 'instruction', 'input', 'system', 'input_ids', 'attention_mask', 'special_tokens_mask', 'labels'],
        num_rows: 594
    })
    test: Dataset({
        features: ['output', 'instruction', 'input', 'system', 'input_ids', 'attention_mask', 'special_tokens_mask', 'labels'],
        num_rows: 66
    })
})

In [11]:
dataset['train'][9]

{'output': '投标人或其他利害关系人认为招标投标活动不符合法律、行政法规规定的，应当自知道或者应当知道之日起**10日内**向有关行政监督部门投诉。若投诉内容涉及招标文件、开标或评标结果，需先按照规定向招标人提出异议，且异议答复期间不计算在10日期限内。投诉需附明确请求和必要的证明材料。',
 'instruction': '投标人或其他利害关系人对招标投标活动有异议时，应在多少天内向行政监督部门投诉？',
 'input': '',
 'system': '',
 'input_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  

In [12]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [13]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="test-output",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    bf16=True,
    num_train_epochs=2,
    push_to_hub=False,
)

In [14]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
)

trainer.train()

It is strongly recommended to train Gemma3 models with the `eager` attention implementation instead of `sdpa`. Use `eager` with `AutoModelForCausalLM.from_pretrained('<path-to-checkpoint>', attn_implementation='eager')`.


Step,Training Loss
500,3.334000
1000,2.595700


TrainOutput(global_step=1188, training_loss=2.87590481536557, metrics={'train_runtime': 523.6405, 'train_samples_per_second': 2.269, 'train_steps_per_second': 2.269, 'total_flos': 1.0187987239305216e+16, 'train_loss': 2.87590481536557, 'epoch': 2.0})

In [15]:
model.save_pretrained("gemma-3-1b-it-full")

## 使用PEFT微调

In [1]:
model_path="/data/download-model/gemma-3-1b-it"

In [2]:
from transformers import AutoModelForCausalLM,AutoTokenizer,TrainingArguments,Trainer

/home/adminad/zhangdw/workspace/uv/transformers/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from peft import get_peft_model,LoraModel, LoraConfig

In [4]:
model = AutoModelForCausalLM.from_pretrained(model_path,
                                             attn_implementation="eager",
                                             torch_dtype="auto", 
                                             device_map="cuda:3")
model

Gemma3ForCausalLM(
  (model): Gemma3TextModel(
    (embed_tokens): Gemma3TextScaledWordEmbedding(262144, 1152, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x Gemma3DecoderLayer(
        (self_attn): Gemma3Attention(
          (q_proj): Linear(in_features=1152, out_features=1024, bias=False)
          (k_proj): Linear(in_features=1152, out_features=256, bias=False)
          (v_proj): Linear(in_features=1152, out_features=256, bias=False)
          (o_proj): Linear(in_features=1024, out_features=1152, bias=False)
          (q_norm): Gemma3RMSNorm((256,), eps=1e-06)
          (k_norm): Gemma3RMSNorm((256,), eps=1e-06)
        )
        (mlp): Gemma3MLP(
          (gate_proj): Linear(in_features=1152, out_features=6912, bias=False)
          (up_proj): Linear(in_features=1152, out_features=6912, bias=False)
          (down_proj): Linear(in_features=6912, out_features=1152, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma3RMSNorm((11

In [19]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer

GemmaTokenizerFast(name_or_path='/data/download-model/gemma-3-1b-it', vocab_size=262144, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<bos>', 'eos_token': '<eos>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'boi_token': '<start_of_image>', 'eoi_token': '<end_of_image>', 'image_token': '<image_soft_token>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<eos>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("<bos>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("<mask>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=False),
	5: AddedToken

In [6]:
peft_config = LoraConfig(task_type="CAUSAL_LM", target_modules=["q_proj", "v_proj"],inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1)

In [7]:
model = get_peft_model(model, peft_config)

In [8]:
model.print_trainable_parameters()

trainable params: 745,472 || all params: 1,000,631,424 || trainable%: 0.0745


In [9]:
from datasets import load_dataset
dataset = load_dataset("json",data_files="/data/dataset/procurement.json")
dataset

DatasetDict({
    train: Dataset({
        features: ['output', 'instruction', 'input', 'system'],
        num_rows: 660
    })
})

In [10]:
dataset = dataset["train"].train_test_split(test_size=0.1, seed=42)
dataset

DatasetDict({
    train: Dataset({
        features: ['output', 'instruction', 'input', 'system'],
        num_rows: 594
    })
    test: Dataset({
        features: ['output', 'instruction', 'input', 'system'],
        num_rows: 66
    })
})

In [11]:
def tokenize_dataset(examples):
    # 编码输入文本（instruction）
    tokenized_inputs = tokenizer(
        examples["instruction"],
        padding="max_length",
        truncation=True,
        max_length=2048,
        return_special_tokens_mask=True,
    )
    
    # 编码目标文本（output）作为labels
    tokenized_labels = tokenizer(
        examples["output"],
        padding="max_length",
        truncation=True,
        max_length=2048,
        return_special_tokens_mask=True,
    )
    
    # 将编码后的output设置为labels
    tokenized_inputs["labels"] = tokenized_labels["input_ids"].copy()
    return tokenized_inputs

In [12]:
dataset = dataset.map(tokenize_dataset, batched=True)
dataset

DatasetDict({
    train: Dataset({
        features: ['output', 'instruction', 'input', 'system', 'input_ids', 'attention_mask', 'special_tokens_mask', 'labels'],
        num_rows: 594
    })
    test: Dataset({
        features: ['output', 'instruction', 'input', 'system', 'input_ids', 'attention_mask', 'special_tokens_mask', 'labels'],
        num_rows: 66
    })
})

In [13]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [14]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="test-output",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    bf16=True,
    num_train_epochs=2,
    push_to_hub=False,
)

In [15]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
)

trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
500,6.388600
1000,4.027500


TrainOutput(global_step=1188, training_loss=4.959068015769676, metrics={'train_runtime': 192.3197, 'train_samples_per_second': 6.177, 'train_steps_per_second': 6.177, 'total_flos': 1.0198869746909184e+16, 'train_loss': 4.959068015769676, 'epoch': 2.0})

In [16]:
model.save_pretrained("gemma-3-1b-it-lora")

## 对比

In [1]:
from transformers import AutoModelForCausalLM,AutoTokenizer
model_path="/data/download-model/gemma-3-1b-it"
model_path1="/home/adminad/zhangdw/workspace/fintuning/transformers/gemma-3-1b-it-full"
model_path2="/home/adminad/zhangdw/workspace/fintuning/transformers/gemma-3-1b-it-lora"

/home/adminad/zhangdw/workspace/uv/transformers/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer=AutoTokenizer.from_pretrained(model_path)
tokenizer

GemmaTokenizerFast(name_or_path='/data/download-model/gemma-3-1b-it', vocab_size=262144, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<bos>', 'eos_token': '<eos>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'boi_token': '<start_of_image>', 'eoi_token': '<end_of_image>', 'image_token': '<image_soft_token>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<eos>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("<bos>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("<mask>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=False),
	5: AddedToken

In [3]:
model1=AutoModelForCausalLM.from_pretrained(model_path1).to("cuda:3")
model1

Gemma3ForCausalLM(
  (model): Gemma3TextModel(
    (embed_tokens): Gemma3TextScaledWordEmbedding(262144, 1152, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x Gemma3DecoderLayer(
        (self_attn): Gemma3Attention(
          (q_proj): Linear(in_features=1152, out_features=1024, bias=False)
          (k_proj): Linear(in_features=1152, out_features=256, bias=False)
          (v_proj): Linear(in_features=1152, out_features=256, bias=False)
          (o_proj): Linear(in_features=1024, out_features=1152, bias=False)
          (q_norm): Gemma3RMSNorm((256,), eps=1e-06)
          (k_norm): Gemma3RMSNorm((256,), eps=1e-06)
        )
        (mlp): Gemma3MLP(
          (gate_proj): Linear(in_features=1152, out_features=6912, bias=False)
          (up_proj): Linear(in_features=1152, out_features=6912, bias=False)
          (down_proj): Linear(in_features=6912, out_features=1152, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma3RMSNorm((11

In [4]:
base_model=AutoModelForCausalLM.from_pretrained(model_path)
from peft import PeftModel
model2=PeftModel.from_pretrained(base_model,model_path2)
model2 = model2.merge_and_unload().to("cuda:3")

In [5]:
inputs="采购中，供应商资质要求都有什么"
inputs_id=tokenizer(inputs,return_tensors="pt").to("cuda:3")
inputs_id

{'input_ids': tensor([[     2,  54843, 237103, 236900,  65432, 172027,  19695,  36144,  10791]],
       device='cuda:3'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:3')}

In [6]:
output1=model1.generate(**inputs_id,max_length=2048)
tokenizer.batch_decode(output1)[0]

/home/adminad/zhangdw/workspace/uv/transformers/lib/python3.11/site-packages/torch/_inductor/compile_fx.py:236: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
  warnings.warn(


'<bos>采购中，供应商资质要求都有什么确保、。规范中。）但，，，平衡需；，，，需。”“，、，”；。，合。，》以。。。合，，”。。采购。合。》。合。化。；“，0。”，。）。，，如。化合。。，风险。、。平衡。。务。。，，。风险。。，。平衡平衡。“平衡。平衡。公平。。。平衡。。。平衡。）。平衡。平衡平衡。平衡。。平衡。平衡，。”。风险”平衡。。风险平衡。。平衡平衡。风险。平衡。平衡。平衡风险。。。平衡性公平平衡平衡平衡。平衡，平衡平衡。。平衡。平衡平衡平衡平衡平衡。平衡。平衡平衡平衡公平平衡平衡平衡平衡平衡平衡。平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡。平衡。平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡。平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡。平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡。。平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡。平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡。平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡。平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平衡平

In [7]:
output2=model2.generate(**inputs_id,max_length=2048)
tokenizer.batch_decode(output2)[0]

'<bos>采购中，供应商资质要求都有什么？\n\n供应商资质要求有几个方面，核心考虑什么关键要素，这些要求对确保供应商提供良好的质量，可靠性、及安全，以及市场价值至关重要。\n\n1.  **资质认证和证书：**\n    *   **ISO 9001:2015**：许多供应商需要获得ISO 9001:2015认证，证明其拥有了质量管理体系。这个认证证明了供应商对质量管理，具有标准化流程和持续改进的承诺。\n    *   **食品安全认证 (FSSC 22001/HACCP)：** 对于食品或食品相关行业的供应商，这两种认证证明了供应商对食品安全和合规性。\n    *   **CE 认证：** 特别是与欧盟市场，需要，需要，需要，针对电子产品和制造产品\n    *   **其他行业认证：** 行业特定证书和认证，比如卫生、特定行业，也很有价值。\n\n2.  **财务和合规：**\n    *   **注册和税务认证：** 验证供应商是否已获得符合所有法律法规的有效注册。\n    *   **付款和财务稳定性：** 评估供应商的财务稳定性。\n    *   **信用审查：**  进行信用历史和历史的全面审查，以验证供应商的信誉。\n\n3.  **供应链管理和运营标准：**\n    *   **供应链绩效：** 衡量供应商如何管理其供应链，这包括他们是否能够跟踪、评估和管理其采购过程\n    *   **安全和环境认证：** 要求供应商遵循负责任的供应链标准，如安全管理（比如，AS/NZS 45001)和环境管理（如ISO 14001）。\n    *   **可追溯性：**  可追溯性，确保供应商可以向客户提供从采购到最终输出，供应链中的可追溯性\n    *   **采购流程：**  供应商与他们的采购流程的标准化和流程的效率评估以及内部的报告\n    *   **供应商 跟踪和沟通:**  对供应商进行对风险、对信誉、对安全管理措施的沟通。\n\n4. **业务能力和经验：**\n    *   **行业知识和经验：** 供应商需要具备您所采购行业，业务领域的经验和知识。\n    *   **能力验证:** 供应商对于具体业务方面的技能、经验和知识可以证明\n    *   **持续改进措施：** 供应商拥有持续改进计划和经验\n\n5. 

In [8]:
base_model=base_model.to("cuda:3")
output3=base_model.generate(**inputs_id,max_length=2048)
tokenizer.batch_decode(output3)[0]

'<bos>采购中，供应商资质要求都有什么？供应商资质管理系统有哪些？\n\n## 供应商资质要求：\n\n供应商资质的要求涵盖了以下几个方面：\n\n1.  **资质校验：**\n    *   **营业执照审核：** 供应商必须具有有效、正本营业执照，用于验证其合法运营。\n    *   **企业资质审核：** 某些行业或采购需求可能需要审核供应商是否拥有相关的资质，例如：\n        *   **税务登记核验：**  有资质认证，如：税务局核实合规、税务信息及相关合规。\n        *   **行业认证：**  某些行业需要认证，例如：采购行业认证和行业合规。\n        *   **专业资质：**  针对特殊行业的资质，例如：有专门的行业审核、质量管理资质等等。\n    *   **资质验证：** 供应商应提供资质文件，并进行官方验证，例如：通过政府网站、第三方机构等。\n    *   **资质审查：** 评估其资质是否符合企业类型、行业的标准。\n\n2.  **质量管理：**\n    *   **质量管理体系认证：** 供应商拥有ISO9001、ISO14001等质量管理体系认证，证明其质量管理体系的有效性。\n    *   **生产资质/经验：** 尤其对于有特殊产品或生产工艺的供应商，有生产资质或相关的经验，用于核验产品质量和安全。\n    *   **有生产能力，且可提供质量追溯的证明。**\n\n3.  **财务信用：**\n    *   **信用记录：**  供应商的信用记录，包括银行账户、贸易信用记录等。\n    *   **资金实力评估：** 验证供应商是否有足够的资金来满足合同交易。\n\n4.  **安全和合规：**\n    *   **安全许可证/证照：** 某些行业对供应商提供安全许可证或相关证照，例如：危险品运输。\n    *   **环境认证：** 确保供应商符合环保认证。\n    *   **劳动安全合规：** 对供应商进行劳动安全合规调查和评估。\n\n5.  **其他：**\n    *   **信誉调查：** 通过第三方数据库（例如：信用参考报告）进行信誉调查，了解供应商的经营状况和财务状况。\n    *   **信息沟通：** 供应商提供资质、安全证书、经营计划等信息，并进行定